In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
import pandas as pd


def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db


def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234

In [ ]:
rssi = MakeDict( [ MakeDataFromText('CHANGING/05112023 moving/RSSI_30897.csv') ] )
temp = MakeDict( [ MakeDataFromText('CHANGING/05112023 moving/tags_30897.csv') ] )

water = pd.read_csv( 'CHANGING/05112023 moving/water temp 05-11-23.csv', skiprows=0, sep=',', header=0 )
water.columns = [ 'Timestamp', 'Serial Number', 'Channel', 'Data', 'Unit of Measure' ]
water = water.iloc[:72]

s1_time, s1_data, s2_time, s2_data = [], [], [], []
for i in range(72):
    _,hms = water['Timestamp'][i].split(' ')
    h,m = hms.split(':')
    time = datetime.datetime(1970, 2, 1, int(h[-2:]), int(m), 0)

    if water['Channel'][i]==1:
        s1_data.append(water['Data'][i])
        s1_time.append(time)
    
    elif water['Channel'][i]==2:
        s2_data.append(water['Data'][i])
        s2_time.append(time)

_s1t, _s2t = [ (j-s1_time[0]).total_seconds() for j in s1_time ], [ (j-s2_time[0]).total_seconds() for j in s2_time ]
models1 = np.poly1d( np.polyfit(_s1t, s1_data,  2) )
models2 = np.poly1d( np.polyfit(_s2t, s2_data,  2) )
strs1 = ", " + str(round(models1[2], 11))+"x^2 + "+str(round(models1[1], 5))+"x + "+str(round(models1[0], 3))
strs2 = ", " + str(round(models2[2], 11))+"x^2 + "+str(round(models2[1], 5))+"x + "+str(round(models2[0], 3))
y2_s1, y2_s2 = r2_score(s1_data, models1(_s1t)), r2_score(s2_data, models2(_s2t))

In [ ]:
fig, (ax1,ax2) = plt.subplots(2, 1, figsize=(16, 12), sharex=True)
fig.subplots_adjust(hspace=0.3)
axs = [ ax1, ax2 ]
for a__ in axs:
    a__.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
    a__.tick_params(labelbottom=True)

TAGS = [ '2B42', '3242', '3943' ];

for i in range(2):
    for k,v in temp.items():
        if k[-4:]==TAGS[i]:
            st, sr = sorted(temp[k].items()), sorted(rssi[k].items())
            dt = [ j[1] for j in st if j[0] > datetime.datetime(1970, 2, 1, 16, 54, 59) ];
            tt = [ j[0] for j in st if j[0] > datetime.datetime(1970, 2, 1, 16, 54, 59) ];
            dr = [ j[1] for j in sr if j[0] > datetime.datetime(1970, 2, 1, 16, 54, 59) ];

            actual = []
            new_tt = []
            for p in range(len(dr)):
                if dr[p]<16:
                    if i==0:    act = 0.032 * (9.83  - float(dr[p])) + float(dt[p]) + 0.6368;
                    elif i==1:  act = 0.037 * (22.53 - float(dr[p])) + float(dt[p]) + 0.5043;
                    elif i==2:  act = 0.048 * (2.84  - float(dr[p])) + float(dt[p]) + 1.0122;
                    actual.append(act)
                    new_tt.append(tt[p])

            T = [ (u-s1_time[0]).total_seconds() for u in new_tt ]

            A, W, R, RSSI_ = [], [], [], []
            for j in range(len(T)):
                A.append( actual[j] )
                W.append( (models1(T[j]) + models2(T[j]))/2.0 )
                R.append( dt[j] )
                RSSI_.append(dr[j])
            A, W, R, RSSI_ = np.array(A), np.array(W), np.array(R), np.array(RSSI_)

            A_Diff = W-A
            R_Diff = W-R

            axs[i].plot(new_tt, A_Diff, label="Water Minus Actual", color="#ED010C", linewidth=0.75 )
            axs[i].plot(new_tt, R_Diff, label="Water Minus Raw",    color="#02DB2A", linewidth=0.75 )
            axs[i].hlines(0, new_tt[0], new_tt[-1], color='black', linewidth=1.0)

            axs[i].legend()
            axs[i].set_ylabel('Temperature (C)', fontsize=13)
            axs[i].set_xlabel('Time (s)', fontsize=13)
            axs[i].set_title('Difference from Water, Tag:' + TAGS[i] + ", RSSI<16, Moving Reader", fontsize=14)